In [1]:
from micrograd.engine import Tensor
import micrograd.nn as nn
import micrograd.functional as F
from micrograd.optim import SGD
import numpy as np
from tqdm import tqdm

In [2]:
# use torchvision to get mnist data
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

CNN definition

In [3]:
class CNN(nn.Module):
    def __init__(self):
        self.conv1 = nn.Conv2d(1, 6, K=5, stride=2, padding=2)
        self.conv2 = nn.Conv2d(6, 16, K=5, stride=2)

        self.lin1 = nn.Linear(400, 120)
        self.lin2 = nn.Linear(120, 80)
        self.lin3 = nn.Linear(80, 10)

    def __call__(self, x):
        x = self.conv1(x).relu()
        x = self.conv2(x).relu()
        x = x.reshape((-1, 400))
        x = self.lin1(x).relu()
        x = self.lin2(x).relu()
        x = self.lin3(x)
        return x

Train

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [5]:
model = CNN()
optimizer = SGD(model.parameters(), lr=0.1)

max_epochs = 1
for epoch in range(max_epochs):
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for it, (x, y) in pbar:
        x = Tensor(x.permute(0, 2, 3, 1).numpy())
        y = y.numpy()
        optimizer.zero_grad()
        ypred = model(x)
        loss = F.batched_cross_entropy(ypred, y)
        loss.backward()
        optimizer.step()
        
        pbar.set_description(f"epoch: {epoch}, it: {it}, loss: {loss.data.item():.5f}")

epoch: 0, it: 1874, loss: 0.01208: 100%|████| 1875/1875 [01:14<00:00, 25.09it/s]


Test

In [6]:
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                          download=True)
test_loader = DataLoader(test_dataset, batch_size=128)

In [7]:
correct, total = 0, 0
for x, y in test_loader:
    x = Tensor(x.permute(0, 2, 3, 1).numpy())
    y = y.numpy()
    logits = model(x)
    pred = np.argmax(logits.data, axis=-1)
    correct += (y == pred).sum()
    total += x.data.shape[0]

print(f"accuracy: {correct / total}")

accuracy: 0.9708
